In [1]:
import pickle
import sys
import timeit

import numpy as np
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import import_ipynb

from sklearn.metrics import mean_squared_error, precision_score, recall_score
from emetrics import get_aupr, get_cindex, get_rm2
from keras.preprocessing import text, sequence

import pandas as pd
import torch
from torch.nn import Sequential, Linear, ReLU

import copy
from torch.autograd import Variable

importing Jupyter notebook from emetrics.ipynb


<string>:34: SyntaxWarning: "is not" with a literal. Did you mean "!="?


In [43]:

class CompoundProteinInteractionPrediction(nn.Module):
    def __init__(self):
        super(CompoundProteinInteractionPrediction, self).__init__()
        
        
        ### hyperparameters:        
        self.n_encoder=3
        self.heads=2
        self.dropout = 0
        self.dim=100
        self.d_ff=10
        
        
        # predefined word embedding
        self.embed_word = nn.Embedding(n_word, self.dim)
        self.embed_word.weight = nn.Parameter(torch.tensor(pro_embedding_matrix, dtype=torch.float32))
        self.embed_word.weight.requires_grad = True
        
 
        self.embed_smile = nn.Embedding(n_smiles, self.dim)
        self.embed_smile.weight = nn.Parameter(torch.tensor(smi_embedding_matrix, dtype=torch.float32))
        self.embed_smile.weight.requires_grad = True

        
        # transformer:
        self.protein_positional_encoder=protein_positional_encoder(self.dim,self.dropout,n_protein)
        self.smiles_positional_encoder=smiles_positional_encoder(self.dim,self.dropout,n_smiles)
        self.encoder=encoder(self.n_encoder, self.dim, self.d_ff, self.dropout, heads=self.heads)

    
    def transformer_protein(self, protein, n_encoder, heads):
        protein=self.protein_positional_encoder(protein)
        protein=self.encoder(protein)
        return protein
    
    def transformer_smiles(self, smiles, n_encoder, heads):
        smiles=self.smiles_positional_encoder(smiles)
        smiles=self.encoder(smiles)
        return smiles
    

    def forward(self, inputs):

        proteins, smiles = inputs


        """Protein vector with transformer."""
        
        p_vectors = self.embed_word(proteins)
        s_vectors=self.embed_smile(smiles)
        
        protein_vector = self.transformer_protein(p_vectors,self.n_encoder,self.heads)
        smiles_vector=self.transformer_smiles(s_vectors,self.n_encoder,self.heads)

        """Concatenate the above two vectors and output the interaction."""
        cat_vector = torch.cat((smiles_vector, protein_vector), 1)
        interaction = self.W_interaction(cat_vector)

        return interaction

    
    def __call__(self, data, train=True):

        inputs, correct_interaction = data[:-1], data[-1]
        predicted_interaction = self.forward(inputs)
        predicted_interaction = torch.squeeze(predicted_interaction, 0)
        if train:
            loss_fuc = torch.nn.MSELoss()
            loss = loss_fuc(predicted_interaction, correct_interaction)
            return loss
        else:
            correct_labels = correct_interaction.to('cpu').data.numpy()
            # ys = F.softmax(predicted_interaction, 1).to('cpu').data.numpy()
            ys = predicted_interaction.to('cpu').data.numpy()
            # predicted_labels = list(map(lambda x: np.argmax(x), ys))
            # predicted_scores = list(map(lambda x: x[1], ys))
            predicted_scores = ys
            return correct_labels, predicted_scores

# end of the model
        

def clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])
  
    
class protein_positional_encoder(nn.Module):
    def __init__(self, d_model, dropout, max_len):
        super(protein_positional_encoder, self).__init__()
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp((torch.arange(0, d_model, 2)).type(torch.FloatTensor) *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position.type(torch.FloatTensor) * div_term.type(torch.FloatTensor))
        pe[:, 1::2] = torch.cos(position.type(torch.FloatTensor) * div_term.type(torch.FloatTensor))
        self.register_buffer('pe', pe)
        self.dropout = nn.Dropout(p=dropout)
    def forward(self, x):
        x = x + Variable(self.pe[0:x.size(0), :x.size(1)], 
                         requires_grad=False)
        return self.dropout(x)
    
    
    
class smiles_positional_encoder(nn.Module):
    def __init__(self, d_model, dropout, max_len):
        super(smiles_positional_encoder, self).__init__()
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp((torch.arange(0, d_model, 2)).type(torch.FloatTensor) *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position.type(torch.FloatTensor) * div_term.type(torch.FloatTensor))
        pe[:, 1::2] = torch.cos(position.type(torch.FloatTensor) * div_term.type(torch.FloatTensor))
        self.register_buffer('pe', pe)
        self.dropout = nn.Dropout(p=dropout)
    def forward(self, x):
        x = x + Variable(self.pe[0:x.size(0), :x.size(1)], 
                         requires_grad=False)
        return self.dropout(x)




class encoder(nn.Module):
    def __init__(self, n, dim, d_ff, dropout, heads):
        super(encoder, self).__init__()
        self.layers = clones(encoder_layer(dim, heads, 
                             self_attn(heads, dim, dropout).to(device), 
                             PositionwiseFeedForward(dim, d_ff), dropout), n)
        
    def forward(self, x, mask=None): 
        for layer in self.layers:
            x = layer(x, mask)
        return x





## attentions:
class self_attn(nn.Module):
    def __init__(self, h, dim, dropout=0):
        super(self_attn, self).__init__()
        assert dim % h == 0
        # We assume d_v always equals d_k
        self.d_k = dim // h
        self.h = h
        self.linears = clones(nn.Linear(dim, dim), 3)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, query, key, value, mask=None): 
        if mask is not None:
            mask = mask.unsqueeze(1)
        nwords = key.size(0) 
      
        query, key, value = \
            [l(x).view(nwords, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))]
        # qkv.size() = length,heads,1,dk
            
        query=query.squeeze(2).transpose(0,1)               # heads, length, dk
        key=key.squeeze(2).transpose(0,1).transpose(1,2)    # heads, dk, length
        value=value.squeeze(2).transpose(0,1)               # heads, length, dk
        
        scores = torch.matmul(query,key)                    # heads, length, length
        p_attn = F.softmax(scores, dim = 2)                 # heads, length, length
        if self.dropout is not None:
            p_attn = self.dropout(p_attn)
        
        x=torch.matmul(p_attn, value)                       # heads, length, dk
        x=x.transpose(0,1).contiguous().view([nwords,self.h * self.d_k]) 
        #x=x.transpose(0,1).view([nwords,self.h * self.d_k]) 
        self.attn=p_attn  
        
        return self.linears[-1](x).unsqueeze(1)  
    
    
    


class encoder_layer(nn.Module):
    def __init__(self, dim, heads, self_attn, feedforward, dropout):  
        super(encoder_layer, self).__init__()
        self.res_layer = [residual_layer( dim, dropout,self_attn),
                          residual_layer( dim, dropout,feedforward)]
        self.dim=dim
    def forward(self, x, mask=None):
        x = self.res_layer[0](x,x,x)
        return self.res_layer[1](x)
    
    



class residual_layer(nn.Module):
    def __init__(self, size, dropout, sublayer):
        super(residual_layer, self).__init__()
        self.norm = LayerNorm(size).to(device)
        self.dropout = nn.Dropout(dropout)
        self.sublayer=sublayer
        
    def forward(self,x, q=None, k=None ):  # q and k are None if sublayer is ff, x is v
        if (q!=None and k!=None):
            return self.norm(x+self.dropout(self.sublayer(q,k,x).squeeze(1)))
        else:
            return self.norm(x+self.dropout(self.sublayer(x).squeeze(1)))
        
        
        
    
    
class LayerNorm(nn.Module):
    def __init__(self, size, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(size))
        self.b_2 = nn.Parameter(torch.zeros(size))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        #return norm+bias
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2   
    
    
    
    
class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff).to(device)
        self.w_2 = nn.Linear(d_ff, d_model).to(device)
        self.dropout = nn.Dropout(dropout).to(device)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))



        
class Trainer(object):
    def __init__(self, model):
        self.model = model
        self.optimizer = optim.Adam(self.model.parameters(),
                                    lr=2e-4, weight_decay=0)

    def train(self, dataset):
        np.random.shuffle(dataset)
        N = len(dataset)
        loss_total = 0
        for data in dataset[0]:
            loss = self.model(data)
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            loss_total += loss.to('cpu').data.numpy()
        return loss_total


class Tester(object):
    def __init__(self, model):
        self.model = model

    def test(self, dataset):
        N = len(dataset)
        print(N)
        T, S = [], []
        for data in dataset[0]:
            (correct_labels,predicted_scores) = self.model(data, train=False)
            # print(correct_labels, predicted_scores)
            T.append(correct_labels)
            S.append(predicted_scores)
        MSE = mean_squared_error(T, S)
        cindex = get_cindex(T, S)
        rm2 = get_rm2(T, S)
        AUPR = get_aupr(T, S)
        return MSE, cindex, rm2, AUPR, T, S

    def save_MSEs(self, MSEs, filename):
        with open(filename, 'a') as f:
            f.write('\t'.join(map(str, MSEs)) + '\n')
            
    def save_predictions(self, predictions, filename):
        with open(filename, 'w') as f:
            f.write('Predict\n')
            f.write(str(predictions))
            
    def save_model(self, model, filename):
        torch.save(model.state_dict(), filename)


def load_tensor(file_name, dtype):
    return [dtype(d).to(device) for d in np.load(file_name + '.npy', allow_pickle = True)]


def load_pickle(file_name):
    with open(file_name, 'rb') as f:
        return pickle.load(f)
    
def shuffle_dataset(dataset, seed):
    np.random.seed(seed)
    np.random.shuffle(dataset)
    return dataset


def split_dataset(dataset, ratio):
    n = int(ratio * len(dataset))
    dataset_1, dataset_2 = dataset[:n], dataset[n:]
    return dataset_1, dataset_2



def w2v_pad(protein, victor_size):
    

    protVec_model = {}
    words=[]
    with open("C:/Users/park/Desktop/Drugdiscovery/dataset/embed/protVec_100d_3grams.csv", encoding='utf8') as f:
        for line in f:
            values = eval(line).rsplit('\t')
            word = values[0]
            words.append(word)
            coefs = np.asarray(values[1:], dtype='float32')
            protVec_model[word] = coefs
        
    print("add protVec finished....")

    count=0
    count_random=0
    embedding_matrix =[]
    
    for word in protein:
        protein_3gram=[]
        sequence_matrix =[]
        count_sequence=0
        for i in range(len(word)-2):
            protein_gram=word[i:i+3]
            protein_3gram.append(protein_gram)
            if protein_gram in protVec_model:
                sequence_matrix.append(protVec_model[protein_gram])
                count_sequence+=1
                count+=1
            else:
                unk_vec = np.random.random(victor_size) * 0.5
                unk_vec = unk_vec - unk_vec.mean()
                sequence_matrix.append(unk_vec)
                count_sequence+=1
                count_random+=1
                
            if count_sequence==998:
                break
               
        if len(word)<998:
            zero_pad=np.zeros([998-len(word),100])
            sequence_matrix=np.append(sequence_matrix,zero_pad,axis=0)
        
        
        embedding_matrix.append(sequence_matrix)
        
    
    print(count)
    print(count_random)
    
        
    return embedding_matrix





def smile_w2v_pad(smile, victor_size):


    smileVec_model = {}
    words=[]
    
    with open("C:/Users/park/Desktop/Drugdiscovery/dataset/embed/Atom.txt", encoding='utf8') as f:
        for line in f:
            values = line.rstrip().rsplit(' ')
            word = values[0]
            words.append(word)
            coefs = np.asarray(values[1:], dtype='float32')
            coefs
            smileVec_model[word] = coefs
            
    
    print("add smileVec finished....")
    
    count=0
    count_random=0
    embedding_matrix=[]
    
    print(len(smile))
    for word in smile:
        word=word.split()
        smile_n_gram=[]
        smile_matrix=[]
        count_sequence=0
        for i in range(len(word)-2):
            smile_gram=word[i]
            smile_n_gram.append(smile_gram)
            if smile_gram in smileVec_model:
                smile_matrix.append(smileVec_model[smile_gram])
                count_sequence+=1
                count+=1
            elif smile_gram==' ':
                pass
            else:
                print(smile_gram)
                unk_vec = np.random.random(victor_size) * 0.5
                unk_vec = unk_vec - unk_vec.mean()
                smile_matrix.append(unk_vec)
                count_random+=1
                count_sequence+=1
                
            if count_sequence==100:
                break
                
        if len(word)<100:
            zero_pad=np.zeros([100-len(word),100])
            smile_matrix=np.append(smile_matrix,zero_pad,axis=0)
            
        embedding_matrix.append(smile_matrix)
    
    print(count)
    print(count_random)
                
    
    
    return embedding_matrix


if __name__ == "__main__":
    
    """CPU or GPU."""
    if torch.cuda.is_available():
        device = torch.device('cuda')
        print('The code uses GPU...')
    else:
        device = torch.device('cpu')
        print('The code uses CPU!!!')
        
    """processing protein sequence"""
    with open("C:/Users/park/Desktop/Drugdiscovery/dataset/Davis/proteins.txt", 'r') as f:
        p=f.readlines()
        pp=p[0].split(":")
        
        protein=[]
        for i in pp:
            str=i.split(",")
            value=str[0].strip()
            protein.append(value[1:len(value)-1])
        protein.pop(0)



    """processing SMILES"""
    with open("C:/Users/park/Desktop/Drugdiscovery/dataset/smile_n_gram.txt", 'r') as f:
        smile = f.read().strip().split('\n')

    
        
    """Load preprocessed data."""
    dir_input = ("C:/Users/park/Desktop/Drugdiscovery/dataset/Davis/inputtest/")
    
    
    pro_embedding_matrix = w2v_pad(protein, 100)
    np.save(dir_input+"protein.npy", pro_embedding_matrix)
    
    smi_embedding_matrix = smile_w2v_pad(smile, 100)
    np.save(dir_input+"smile.npy", smi_embedding_matrix)
    
    pro_embedding_matrix=np.array(pro_embedding_matrix)
    smi_embedding_matrix=np.array(smi_embedding_matrix)
    
    interactions = load_tensor(dir_input + 'interactions', torch.FloatTensor)
    Y = load_tensor(dir_input + 'interactions', torch.FloatTensor)
    word_dict = load_pickle(dir_input + 'word_dict.pickle')
    
    n_word=len(word_dict)
    n_smiles=79

    
    

    """data preprocessing"""
    Y = np.asarray(Y)
    interactions = -(np.log10(Y / (math.pow(10, 9))))
    interactions = list(interactions)
    

    """Create a dataset and split it into train/dev/test."""
    dataset = list(zip(pro_embedding_matrix, smi_embedding_matrix, interactions))
    print(len(dataset))
    dataset = shuffle_dataset(dataset, 1234)


    dataset_train, dataset_ = split_dataset(dataset, 2/3)
    dataset_dev, dataset_test = split_dataset(dataset_, 0.5)

    
    """Set a model."""
    torch.manual_seed(1234)
    model = CompoundProteinInteractionPrediction().to(device)

    trainer = Trainer(model)
    tester = Tester(model)

    """Output files."""
    file_MSEs = "C:/Users/park/Desktop/Drugdiscovery/output/result/Transformer.txt"
    file_T = "C:/Users/park/Desktop/Drugdiscovery/output/predict/T--Transformer.txt"
    file_S = "C:/Users/park/Desktop/Drugdiscovery/output/predict/S--Transformer.txt"
    file_model = "C:/Users/park/Desktop/Drugdiscovery/output/model/Transformer"
    MSEs = ('Epoch\tTime(sec)\tLoss_train\tMSE_dev\t'
            'MSE_test\tciindex_test\trm2\tAUPR')
    with open(file_MSEs, 'w') as f:
        f.write(MSEs + '\n')

    """Start training."""
    print('Training...')
    print(MSEs)
    start = timeit.default_timer()

    for epoch in range(1, 100):
        if epoch % 10 == 0:
            trainer.optimizer.param_groups[0]['lr'] *= 0.5

        loss_train = trainer.train(dataset_train)
        MSE_dev = tester.test(dataset_dev)[0]
        MSE_test, cindex, rm2, AUPR, T, S = tester.test(dataset_test)

        end = timeit.default_timer()
        time = end - start

        MSEs = [epoch, time, loss_train, MSE_dev,
                MSE_test,cindex, rm2, AUPR]
        tester.save_MSEs(MSEs, file_MSEs)
        tester.save_model(model, file_model)
        tester.save_predictions(T, file_T)
        tester.save_predictions(S, file_S)
        print('\t'.join(map(str, MSEs)))

The code uses CPU!!!
add protVec finished....
313934
0
add smileVec finished....
30056


KeyboardInterrupt: 